In [48]:
import pandas as pd
import plotly.express as px
import plotly.graph_objs as go
import plotly.io as pio

from spotpy.objectivefunctions import rmse, nashsutcliffe, pbias, kge
from mogestpy.quantity.Hydrological.MassBalance import MassBalance

pd.options.plotting.backend = "plotly"
pio.templates.default = "plotly_white"

In [49]:
df = pd.read_excel('Qjurumirim.xlsx')

In [50]:
df_calc = pd.read_excel('qtotal.xlsx', index_col=0)

In [51]:
df_calc.plot()

In [52]:
VU = 2860 # 1e6 m3

In [53]:
df['VU'] = df.Vuperc * VU / 100

In [54]:
Vi = df['VU'][0]

In [55]:
df

,Qafl,Qnat,Qdef,Vuperc,VU
0,150.47,183.95,188,66.48,1901.328
1,178.62,186.64,188,66.46,1900.756
2,205.00,230.45,205,66.46,1900.756
3,209.00,228.16,209,66.46,1900.756
4,188.00,186.61,188,66.46,1900.756
...,...,...,...,...,...
361,184.05,183.38,209,29.92,855.712
362,167.44,174.11,209,29.81,852.566
363,179.63,183.19,163,29.85,853.710
364,203.57,196.73,162,29.97,857.142


In [56]:
df['MB1'] = MassBalance(df.Qafl, df.Qdef, 86400, Vi * 1e6).Volume_out()
df['MB2'] = MassBalance(df.Qafl * 1.05, df.Qdef, 86400, Vi * 1e6).Volume_out()
df['MB3'] = MassBalance(df_calc.SMAP, df.Qdef, 86400, Vi * 1e6).Volume_out()

In [57]:
df['MB1'] = df['MB1'].div(1e6)
df['MB2'] = df['MB2'].div(1e6)
df['MB3'] = df['MB3'].div(1e6)

In [63]:
fig = px.line(df, x=df.index, y=['MB1', 'MB2', 'MB3', 'VU'])
fig.update_traces(line=dict(dash='dash', color='black'), selector=dict(name='VU'))

fig.update_traces(name='Referência', selector=dict(name='VU'))
fig.update_traces(name='Calculado - Dados Originais', selector=dict(name='MB1'))
fig.update_traces(name='Calculado - Vazão afluente 5% maior', selector=dict(name='MB2'))
fig.update_traces(name='Vazões SMAP', selector=dict(name='MB3'))

fig.update_layout(
    xaxis_title='Tempo (dias)',
    yaxis_title='Volume Útil (10^6 m³)',
    font=dict(family='CMU Serif', size=13),
    width=700,
    height=500,
    legend_title='Legenda',
    legend=dict(
        orientation='h',
        itemsizing='constant',
    ),
    margin=dict(l=0, r=10, t=0, b=0)
)
fig.show()


In [59]:
fig.write_image("MB.pdf")

In [60]:
df['MB1perc'] = df['MB1'] / df['VU'] * 100
df['MB2perc'] = df['MB2'] / df['VU'] * 100

In [61]:
erro = rmse(df['Vuperc'], df['MB1perc'])
percbias = pbias(df['Vuperc'], df['MB1perc']) * 100

print(f'RMSE: {erro}')  
print(f'PBIAS: {percbias}')

RMSE: 37.23847282982702
PBIAS: 5596.661134503484


In [62]:
erro = rmse(df['Vuperc'], df['MB2perc'])
percbias = pbias(df['Vuperc'], df['MB2perc']) * 100

print(f'RMSE: {erro}')  
print(f'PBIAS: {percbias}')

RMSE: 54.05479656207175
PBIAS: 7772.759083202888
